In [1]:
import requests #https://curl.trillworks.com/#python
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import csv

# Author: Renzo Caballero
# KAUST: King Abdullah University of Science and Technology
# email 1: renzo.caballerorosas@kaust.edu.sa
# email 2: CaballeroRenzo@hotmail.com
# email 3: CaballeroRen@gmail.com
# Website: None
# November 2019; Last revision: 15/01/2020

# For more information about this file, check ./Notes/generalNotes.pdf.

In [2]:
def Wind_G_F(Date, Parques, Procedencia, Hora, Tipo): #https://apps.ute.com.uy/SgePublico/ConsPrevGeneracioEolica.aspx
    
    # On 14/01/2020, I updated this cURL w.r.t. the older versions.
    # I do not know what is different, but it seems to work after I made the change.
    
    cookies = {
    'ASP.NET_SessionId': 'rtkkpwvns3xd24bez0rshd55',
    }

    headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Origin': 'https://apps.ute.com.uy',
    'Upgrade-Insecure-Requests': '1',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
    'Sec-Fetch-User': '?1',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Referer': 'https://apps.ute.com.uy/SgePublico/ConsPrevGeneracioEolica.aspx',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
    }

    data = {
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__LASTFOCUS': '',
    'ctl00_TreeViewMain_ExpandState': 'nnnnnnnnnnnnnnnnnnnnnnnnn',
    'ctl00_TreeViewMain_SelectedNode': '',
    'ctl00_TreeViewMain_PopulateLog': '',
    '__VIEWSTATE': '/wEPDwUKMjEwNTI2MjQ4MQ9kFgJmDw8WAh4JX09wY2lvbmVzBQdFTkVSR0lBZBYCAgMPZBYGAgEPDxYCHgRUZXh0BSJQcm9uw7NzdGljbyBkZSBHZW5lcmFjacOzbiBFw7NsaWNhZGQCDQ9kFhYCAw8QZBAVEApBcnRpbGxlcm9zC0NhcmFwZSBJK0lJCUNhcmFjb2xlcxNDLlBlcmFsdGErUGVyMStQZXIyEUXDs2xpY28gRmxvcmlkYSBJEEp1YW4gUGFibG8gVGVycmEIS2VudGlsdXgLTHVjZXMgUitMK00HTWluYXMgSQhNZWxvd2luZAVQYW1wYQlSIGRlbCBTdXIVVGFsYXMgZGVsIE1hY2llbCBJK0lJFlRvdGFsIHBhw61zIChwcm92aXN0bykKVmFsZW50aW5lcxZWZW50dXMgQ29tLiBFbmVyLiBTLkEuFRAKQVJUSTAxUE9UUApDQVBFMDFQT1RQCkNBUkEwMVBPVFAKQ1BQUDAxUE9UUApGTE8xMDFQT1RQCkpQVEUwMVBPVFAKS0VOVDAxUE9UUApMUkxNMDFQT1RQCk1JTjEwMVBPVFAKTVdJTjAxUE9UUApQQU1QMDFQT1RQClJTVVIwMVBPVFAKVERNQTAxUE9UUApVUlVZMDFQT1RQClZBTEUwMVBPVFAKVkVDTzAxUE9UUBQrAxBnZ2dnZ2dnZ2dnZ2dnZ2dnFgECDWQCBw8QZGQWAWZkAgsPEGRkFgFmZAIRDzwrABABABQrAAMPFgIeClBvc3RlZERhdGUFCjE0LzAxLzIwMjBkZGQWBAIDDw8WAh4NQWx0ZXJuYXRlVGV4dAUFIC4uLiAWBB4Fc3R5bGUFKnZlcnRpY2FsLWFsaWduOnRleHQtYm90dG9tO2N1cnNvcjpwb2ludGVyOx4Hb25jbGljawVIQ2FsZW5kYXJQb3B1cF9GaW5kQ2FsZW5kYXIoJ2N0bDAwX0NvbnRlbnRQbGFjZUhvbGRlcjFfRmVjaGFJbmknKS5TaG93KCk7ZAIEDxYCHgV2YWx1ZQUKMjMvMDQvMjAxOWQCFQ8QZBAVAwxBV1NUcnVlcG93ZXIMTWV0ZW9sw7NnaWNhL1VURSBwcm9ub3N0aWNvIDUgLWVuc2VtYmxlcy0gaGF6IGRlIDIwIGNvcnJpZGFzFQMFQVdTVFAFTVRMT0cFVVRFUDUUKwMDZ2dnFgFmZAIZDxBkEBUDBGgyODgCaDYDaDcyFQMEaDI4OAJoNgNoNzIUKwMDZ2dnFgECAmQCHQ8QZBAVBAUwMTowMAUwNzowMAUxMzowMAUxOTowMBUEBDAxMDAEMDcwMAQxMzAwBDE5MDAUKwMEZ2dnZxYBZmQCHw8PFgIeB0VuYWJsZWRoZGQCIw88KwANAQAPFgYeC18hRGF0YUJvdW5kZx4LXyFJdGVtQ291bnQCAR4HVmlzaWJsZWhkFgJmD2QWBAIBD2QWDGYPDxYCHwEFBiZuYnNwO2RkAgEPDxYCHwEFBiZuYnNwO2RkAgIPDxYCHwEFBiZuYnNwO2RkAgMPDxYCHwEFBiZuYnNwO2RkAgQPDxYCHwEFBiZuYnNwO2RkAgUPDxYCHwEFBiZuYnNwO2RkAgIPDxYCHwpoZGQCKQ8PFgQfAQVATm8gc2UgZGlzcG9uZSBkZSBwcm9uw7NzdGljbyBwYXJhIGxhcyBjb25kaWNpb25lcyBzZWxlY2Npb25hZGFzLh8KaGRkAisPDxYEHwEFd1NlIGNvbnNpZGVyYSBsYSBkaXNwb25pYmlsaWRhZCBkZWwgMTAwICUgZGUgbGEgcG90ZW5jaWEgYXV0b3JpemFkYSBkZSBsYXMgdW5pZGFkZXMgZ2VuZXJhZG9yYXMgc2VsZWNjaW9uYWRhcyAgKDE0ODggTVcpHwpoZGQCDw88KwAJAgAPFgYeDU5ldmVyRXhwYW5kZWRkHgxTZWxlY3RlZE5vZGVkHglMYXN0SW5kZXgCGWQIFCsAGgVPMTo0MCwxOjM5LDE6MzgsMTozNywxOjM2LDE6MzUsMTozNCwxOjMzLDE6MzIsMTozMSwxOjMwLDE6MjksMToyOCwxOjI3LDE6MjYsMToyNRQrAAIWAh4IRXhwYW5kZWRoZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZBQrAAIWAh8OaGQUKwACFgIfDmhkFCsAAhYCHw5oZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgEFEmN0bDAwJFRyZWVWaWV3TWFpbgUiY3RsMDAkQ29udGVudFBsYWNlSG9sZGVyMSRncmlkUHJvbg88KwAKAQgCAWQaRwQwzHl632ItwPoq0G+ZDK7C5Q==',
    '__VIEWSTATEGENERATOR': '2CA7B51F',
    '__EVENTVALIDATION': '/wEWQALM65y7BwKojr+iBALj3MvgCAL1j//IAQL7iIjPAwL1j9fOCQLP45DTDQL6mNKdDALXzdfxCQKoi7mjBAL9vOP8DwKmr7PzDAKH7pKjCQL9wfN+AouwwsAFAuKrm8YKAo623eALAsvAwpMNAtuD0esHAvzI85UKArTDtCEC17j52AMC1Jf0igoCh8S2xgUC4tTfAwLsnvWEDgL87pOCAgLyutbaCQKrxq7wCwKp/qLSBAKjm9RkAuGR8L8IArH/j7YLAv6a6pgJAtS0o58JAtS0258JAte0q58JAte0g58JAsLE9owOAv2ip9AEArvb85MEArXX6wQCtavwkQ0C/rGGzgECh52z3AQCwdr+UwL9hriSCAKBi6gKApnSmYkJArWo9aoOAt3git8NAu/DvMoGAu7PpOYDApazzdYEAtOSkNcMAsm9hoAJArvi7M4FAraYyrkCAue6oZUMAvuD4PsLAtKLu4cHAtLFkJcIAub8l84EArniqqwKVhXv1BI4TaOPMyORj6uaoQO3kOg=',
    'ctl00$ContentPlaceHolder1$cboParques': Parques,
    'ctl00$ContentPlaceHolder1$cboAlcance': 'Totales',
    'ctl00$ContentPlaceHolder1$cboVariable': 'Potencia',
    'ctl00$ContentPlaceHolder1$cmdAplicar': 'Aplicar',
    'ctl00$ContentPlaceHolder1$FechaIni$textBox': Date, # E.g., '23/04/2019'.
    'ctl00$ContentPlaceHolder1$FechaIni$hidden': Date,
    'ctl00$ContentPlaceHolder1$cboProcedencia': Procedencia,
    'ctl00$ContentPlaceHolder1$cboTipoPron': Tipo,
    'ctl00$ContentPlaceHolder1$cboHoras': Hora,
    'ctl00$ContentPlaceHolder1$hdnCargar': 'NO'
    }

    response = requests.post('https://apps.ute.com.uy/SgePublico/ConsPrevGeneracioEolica.aspx', headers=headers, cookies=cookies, data=data)

    return response

In [3]:
### AWSTP --> 23/04/2019
### MTLOG --> 08/01/2016
### UTEP5 --> 05/07/2017

Date = datetime.datetime(2016,1,8)
final_day = Date.today() - timedelta(days = 6)

weWrote = 1
file_name = 'Wind_Data_MTLOG_1900.csv'
starting_hour = '1900'
Procedencia = 'MTLOG'

while Date < final_day:
    
    if weWrote == 1:
        
        with open(file_name, mode = 'r') as Wind_Data_AWSTP:
            
            Wind_Data_AWSTP = csv.reader(Wind_Data_AWSTP, delimiter = ',')
            List = list(Wind_Data_AWSTP)

            last_date_saved = List[-1][0]
            try:
                date_to_check = datetime.datetime.strptime(last_date_saved, '%d/%m/%Y') - timedelta(days = 3)
            except:
                print('This fails the first time ever we write, because the file does not have any date.')
                date_to_check = datetime.datetime.strptime('01/01/1999', '%d/%m/%Y') 
                
        weWrote = 0

    if  Date > date_to_check:

        weWrote = 1
        
        response = Wind_G_F(Date,'URUY01POTP',Procedencia,starting_hour,'h72')
        error = 0;
        while response.status_code != 200:
            response = Wind_G_F(Date,'URUY01POTP',Procedencia,starting_hour,'h72')
            error = error + 1
            print('Error:', error)

        response_soup = BeautifulSoup(response.content, 'html.parser')
        All_td = response_soup.find_all('td')
        List = list(All_td)
        Table = List[30].table
        All = Table.find_all('td')
        List_all = list(All)
        
        with open(file_name, mode = 'a') as Wind_Data_AWSTP:
            Wind_Data_AWSTP_csv = csv.writer(Wind_Data_AWSTP, delimiter=',')

            for i in range(0,72):
                
                try:
                    print(List_all[i*6+1].string,
                    List_all[i*6+2].string,\
                    starting_hour,
                    List_all[i*6+3].string.replace('.','').replace(',','.'),\
                    List_all[i*6+4].string.replace('.','').replace(',','.'),\
                    List_all[i*6+5].string.replace('.','').replace(',','.'),\
                    List_all[i*6+6].string.replace('.','').replace(',','.'))

                    Wind_Data_AWSTP_csv.writerow([List_all[i*6+1].string,
                    List_all[i*6+2].string,\
                    starting_hour,
                    List_all[i*6+3].string.replace('.','').replace(',','.'),\
                    List_all[i*6+4].string.replace('.','').replace(',','.'),\
                    List_all[i*6+5].string.replace('.','').replace(',','.'),\
                    List_all[i*6+6].string.replace('.','').replace(',','.')])
                except:
                    print('The data does not exist!')
                    
    Date = Date + timedelta(days = 1)

This fails the first time ever we write, because the file does not have any date.
23/04/2019 19:00 1900 358.32 468.47 413.39 604.02
23/04/2019 20:00 1900 403.05 566.20 484.62 610.52
23/04/2019 21:00 1900 413.76 627.69 520.73 622.61
23/04/2019 22:00 1900 404.47 668.10 536.28 644.27
23/04/2019 23:00 1900 381.03 686.93 533.98 635.92
24/04/2019 00:00 1900 333.91 665.35 499.63 627.78
24/04/2019 01:00 1900 296.58 638.50 467.54 594.69
24/04/2019 02:00 1900 261.54 604.98 433.26 480.13
24/04/2019 03:00 1900 220.28 558.83 389.55 401.73
24/04/2019 04:00 1900 187.82 522.44 355.13 373.59
24/04/2019 05:00 1900 161.43 493.48 327.45 373.44
24/04/2019 06:00 1900 154.72 488.87 321.80 372.23
24/04/2019 07:00 1900 160.20 499.47 329.83 379.97
24/04/2019 08:00 1900 163.02 506.82 334.92 291.92
24/04/2019 09:00 1900 170.51 520.09 345.30 226.82
24/04/2019 10:00 1900 197.08 557.40 377.24 246.11
24/04/2019 11:00 1900 219.56 589.76 404.66 266.29
24/04/2019 12:00 1900 242.69 623.12 432.90 318.08
24/04/2019 13:00 1

26/04/2019 19:00 1900 995.72 1106.79 1051.26 1007.44
26/04/2019 20:00 1900 894.10 1058.88 976.49 876.05
26/04/2019 21:00 1900 739.34 966.16 852.75 714.42
26/04/2019 22:00 1900 588.29 881.81 735.05 567.07
26/04/2019 23:00 1900 473.35 796.92 635.13 458.51
27/04/2019 00:00 1900 364.82 702.87 533.84 378.00
27/04/2019 01:00 1900 287.62 627.52 457.57 389.72
27/04/2019 02:00 1900 241.19 579.90 410.55 330.26
27/04/2019 03:00 1900 191.96 523.84 357.90 337.41
27/04/2019 04:00 1900 179.54 512.20 345.87 367.43
27/04/2019 05:00 1900 200.10 541.45 370.78 416.95
27/04/2019 06:00 1900 214.05 562.65 388.35 536.18
27/04/2019 07:00 1900 237.85 596.24 417.05 483.48
27/04/2019 08:00 1900 272.42 643.48 457.95 494.24
27/04/2019 09:00 1900 267.67 641.72 454.70 571.74
27/04/2019 10:00 1900 266.52 644.53 455.53 648.66
27/04/2019 11:00 1900 283.02 669.57 476.30 682.34
27/04/2019 12:00 1900 306.52 703.59 505.06 679.41
27/04/2019 13:00 1900 330.29 738.13 534.21 634.62
27/04/2019 14:00 1900 337.90 752.41 545.15 628

29/04/2019 19:00 1900 263.47 366.99 315.23 328.18
29/04/2019 20:00 1900 249.23 396.88 323.06 415.35
29/04/2019 21:00 1900 241.24 432.54 336.89 476.87
29/04/2019 22:00 1900 247.80 485.96 366.88 574.20
29/04/2019 23:00 1900 268.41 552.75 410.58 580.90
30/04/2019 00:00 1900 289.71 611.70 450.71 622.23
30/04/2019 01:00 1900 320.82 668.22 494.52 657.10
30/04/2019 02:00 1900 363.03 730.05 546.54 781.94
30/04/2019 03:00 1900 371.42 745.57 558.49 883.99
30/04/2019 04:00 1900 380.15 760.58 570.36 896.45
30/04/2019 05:00 1900 392.02 779.60 585.81 921.12
30/04/2019 06:00 1900 386.73 777.38 582.06 929.84
30/04/2019 07:00 1900 390.55 786.56 588.56 859.61
30/04/2019 08:00 1900 396.92 798.99 597.96 737.27
30/04/2019 09:00 1900 375.64 776.90 576.27 828.61
30/04/2019 10:00 1900 359.52 761.22 560.37 763.20
30/04/2019 11:00 1900 329.62 728.17 528.90 599.61
30/04/2019 12:00 1900 289.41 682.01 485.71 509.38
30/04/2019 13:00 1900 254.94 642.92 448.93 464.70
30/04/2019 14:00 1900 229.14 614.66 421.90 397.84


02/05/2019 19:00 1900 384.98 497.00 440.99 758.74
02/05/2019 20:00 1900 420.48 585.38 502.93 849.02
02/05/2019 21:00 1900 465.11 685.77 575.44 894.08
02/05/2019 22:00 1900 497.83 776.64 637.23 949.80
02/05/2019 23:00 1900 509.90 840.46 675.18 1017.33
03/05/2019 00:00 1900 496.66 862.87 679.77 1003.90
03/05/2019 01:00 1900 489.72 875.26 682.49 959.00
03/05/2019 02:00 1900 489.98 886.48 688.23 734.64
03/05/2019 03:00 1900 450.05 842.71 646.38 581.73
03/05/2019 04:00 1900 419.41 809.19 614.30 510.84
03/05/2019 05:00 1900 380.08 764.78 572.43 410.44
03/05/2019 06:00 1900 335.31 713.44 524.38 361.55
03/05/2019 07:00 1900 292.86 664.80 478.83 309.53
03/05/2019 08:00 1900 253.71 620.11 436.91 177.06
03/05/2019 09:00 1900 190.30 544.87 367.58 90.24
03/05/2019 10:00 1900 146.41 493.81 320.11 104.33
03/05/2019 11:00 1900 108.88 450.55 279.72 169.48
03/05/2019 12:00 1900 91.41 432.42 261.91 157.36
03/05/2019 13:00 1900 71.01 410.52 240.77 156.16
03/05/2019 14:00 1900 59.16 399.38 229.27 177.90
03

KeyboardInterrupt: 

In [ ]:
# codes = {

#     # Informational.
#     100: ('continue',),
#     101: ('switching_protocols',),
#     102: ('processing',),
#     103: ('checkpoint',),
#     122: ('uri_too_long', 'request_uri_too_long'),
#     200: ('ok', 'okay', 'all_ok', 'all_okay', 'all_good', '\\o/', '✓'),
#     201: ('created',),
#     202: ('accepted',),
#     203: ('non_authoritative_info', 'non_authoritative_information'),
#     204: ('no_content',),
#     205: ('reset_content', 'reset'),
#     206: ('partial_content', 'partial'),
#     207: ('multi_status', 'multiple_status', 'multi_stati', 'multiple_stati'),
#     208: ('already_reported',),
#     226: ('im_used',),

#     # Redirection.
#     300: ('multiple_choices',),
#     301: ('moved_permanently', 'moved', '\\o-'),
#     302: ('found',),
#     303: ('see_other', 'other'),
#     304: ('not_modified',),
#     305: ('use_proxy',),
#     306: ('switch_proxy',),
#     307: ('temporary_redirect', 'temporary_moved', 'temporary'),
#     308: ('permanent_redirect',
#           'resume_incomplete', 'resume',),  # These 2 to be removed in 3.0

#     # Client Error.
#     400: ('bad_request', 'bad'),
#     401: ('unauthorized',),
#     402: ('payment_required', 'payment'),
#     403: ('forbidden',),
#     404: ('not_found', '-o-'),
#     405: ('method_not_allowed', 'not_allowed'),
#     406: ('not_acceptable',),
#     407: ('proxy_authentication_required', 'proxy_auth', 'proxy_authentication'),
#     408: ('request_timeout', 'timeout'),
#     409: ('conflict',),
#     410: ('gone',),
#     411: ('length_required',),
#     412: ('precondition_failed', 'precondition'),
#     413: ('request_entity_too_large',),
#     414: ('request_uri_too_large',),
#     415: ('unsupported_media_type', 'unsupported_media', 'media_type'),
#     416: ('requested_range_not_satisfiable', 'requested_range', 'range_not_satisfiable'),
#     417: ('expectation_failed',),
#     418: ('im_a_teapot', 'teapot', 'i_am_a_teapot'),
#     421: ('misdirected_request',),
#     422: ('unprocessable_entity', 'unprocessable'),
#     423: ('locked',),
#     424: ('failed_dependency', 'dependency'),
#     425: ('unordered_collection', 'unordered'),
#     426: ('upgrade_required', 'upgrade'),
#     428: ('precondition_required', 'precondition'),
#     429: ('too_many_requests', 'too_many'),
#     431: ('header_fields_too_large', 'fields_too_large'),
#     444: ('no_response', 'none'),
#     449: ('retry_with', 'retry'),
#     450: ('blocked_by_windows_parental_controls', 'parental_controls'),
#     451: ('unavailable_for_legal_reasons', 'legal_reasons'),
#     499: ('client_closed_request',),

#     # Server Error.
#     500: ('internal_server_error', 'server_error', '/o\\', '✗'),
#     501: ('not_implemented',),
#     502: ('bad_gateway',),
#     503: ('service_unavailable', 'unavailable'),
#     504: ('gateway_timeout',),
#     505: ('http_version_not_supported', 'http_version'),
#     506: ('variant_also_negotiates',),
#     507: ('insufficient_storage',),
#     509: ('bandwidth_limit_exceeded', 'bandwidth'),
#     510: ('not_extended',),
#     511: ('network_authentication_required', 'network_auth', 'network_authentication')
    
# }

In [ ]:
# Parques = ['ARTI01POTP','CAPE01POTP','CARA01POTP','CPPP01POTP','FLO101POTP','JPTE01POTP','KENT01POTP',\
#           'LRLM01POTP','MIN101POTP','MWIN01POTP','PAMP01POTP','RSUR01POTP','TDMA01POTP','URUY01POTP',\
#           'VALE01POTP','VECO01POTP']
# Nombres = ['Artilleros','Carape I+II','Caracoles','C.Peralta+Per1+Per2','Eolico Florida I','Juan Pablo Terra',\
#           'Kentilux','Luces R+L+M','Minas I','Melowind','Pampa','R del Sur','Talas del Maciel I+II','Total pais (provisto)',\
#           'Valentines','Ventus Com. Ener. S.A.']
# Procedencia = ['GAHAS','MTLOG','UTEP3','UTEP5']
# TipoPron1 = ['10m432','h240','h6','h72']
# # GAHAS looks super unuseful.
# TipoPron2 = ['h240','h288','h6','h72']
# # h240 and h288 have only 1100. h6 has from 0000 to 2300.
# # h72 has 0100, 0700, 1300 and 1900.
# TipoPron3 = ['h6','h72','h240']
# # h6 has from 0000 to 2300. h72 has 0100, 0700, 1300 and 1900.
# # h240 has only 1100.
# TipoPron4 = ['h240','h288','h6','h72']
# h6_times = ['0000','0100','0200','0300','0400','0500','0600','0700',\
#            '0800','0900','1000','1100','1200','1300','1400','1500',\
#            '1600','1700','1800','1900','2000','2100','2200','2300',]
# h72_times = ['0100','0700','1300','1900']
# h240_times = ['1100']
# h288_times = ['1100']

# # The above is old, the new "Procedencia" are (14/01/2020):

# Procedencia = ['AWSTP','MTLOG','UTEP5']
# # The corresponding starting times are:
# ### AWSTP --> 23/04/2019
# ### MTLOG --> 08/01/2016
# ### UTEP5 --> 05/07/2017